In [2]:
import json
import math
import pytz
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
import csv
from scipy.interpolate import interp1d
from scipy.stats import pearsonr, mode
from scipy.signal import savgol_filter
import xgboost as xgb
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

addressPrefix='C:/Users/sorush.omidvar/Google Drive/Documents/Educational/TAMU/Research/CGM Dataset/Bobak/'
if not os.path.exists(addressPrefix):
    addressPrefix='C:/GDrive/Documents/Educational/TAMU/Research/CGM Dataset/Bobak/'

addDataPrefix='/Users/sorush/My Drive/Documents/Educational/TAMU/Research/Trial/Data/11-5-21-11-15-21'

addUserInput=os.path.join(addDataPrefix,'User inputted')
addHKCM=os.path.join(addDataPrefix,'hk+cm')
addCGM=os.path.join(addDataPrefix,'CGM')
addE4=os.path.join(addDataPrefix,'E4')

exempts=['p2']

pd.options.mode.chained_assignment = None  # default='warn'
plt.style.use({'figure.facecolor':'white'})

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)

In [11]:
if not os.path.exists(os.path.join(addDataPrefix,'All_meals.csv')):
    os.chdir(addUserInput)
    df=[]
    for root, dirs, files in os.walk(addUserInput):
        for file in files:
            if '.csv' in file.lower():
                if 'meals' in file.lower():
                    participantName=file[:file.find('Meals')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp = pd.read_csv(file)
                    dfTemp.insert(0,'Participant',participantName)
                    dfTemp.rename(columns={'startTime':'StartTime'}, inplace=True)
                    dfTemp['StartTime']=pd.to_datetime(dfTemp['StartTime'])
                    dfTemp['FinishTime']=pd.to_datetime(dfTemp['FinishTime'])

                    dfTemp['StartTime']-=pd.DateOffset(hours=5)#fixing the time zone issue
                    dfTemp['FinishTime']-=pd.DateOffset(hours=5)#fixing the time zone issue                    
                    dfTemp.sort_values(["Participant",'StartTime'],ascending = (True, True),inplace=True)
                    if len(dfTemp.columns)!=10:
                        print("MAYDAY. Error in reading csv")
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    print("reading is done")
    dfMeal=df
    
    # dfMeal.insert(2,'startDate',dfMeal['startTime'])
    # dfMeal.insert(4,'finishDate',dfMeal['finishTime'])

    # dfMeal['startTime']=dfMeal['startTime'].dt.time
    # dfMeal['finishTime']=dfMeal['finishTime'].dt.time

    # dfMeal['startTime']= dfMeal['startTime'].dt.hour*3600+dfMeal['startTime'].dt.minute*60+dfMeal['startTime'].dt.second+dfMeal['startTime'].dt.microsecond/1000/1000
    # dfMeal['finishTime']= dfMeal['finishTime'].dt.hour*3600+dfMeal['finishTime'].dt.minute*60+dfMeal['finishTime'].dt.second+dfMeal['finishTime'].dt.microsecond/1000/1000

    # dfMeal['startDate']=pd.to_datetime(dfMeal['startDate']).dt.date
    # dfMeal['finishDate']=pd.to_datetime(dfMeal['finishDate']).dt.date
    
    dfMeal.to_csv(os.path.join(addDataPrefix,'All_meals.csv'),index=False)
else:
    dfMeal=pd.read_csv(os.path.join(addDataPrefix,'All_meals.csv'))
    dfMeal['StartTime']=dfMeal['StartTime'].astype('datetime64[ns]')
    dfMeal['FinishTime']=dfMeal['FinishTime'].astype('datetime64[ns]')

In [12]:
if not os.path.exists(os.path.join(addDataPrefix,'All_cgm.csv')):
    os.chdir(addCGM)
    df=[]
    for root, dirs, files in os.walk(addCGM):
        for file in files:
            if '.csv' in file.lower():
                if 'processed' in file.lower():
                    participantName=file[:file.find('Processed')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp = pd.read_csv(file)
                    dfTemp.insert(0,'Participant',participantName)
                    dfTemp['Time']=pd.to_datetime(dfTemp['Time'])
                    dfTemp.sort_values(["Participant","Time"],ascending = (True,True),inplace=True)
                    if len(dfTemp.columns)!=4:
                        print("MAYDAY. Error in reading csv")
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    print("reading is done")
    dfCGM=df
    
    # dfCGM.insert(1,'Date',dfCGM['Time'])
    # dfCGM['Date']=pd.to_datetime(dfCGM['Date']).dt.date
    # dfCGM['Time']=pd.to_datetime(dfCGM['Time'])
    # dfCGM['Time']= dfCGM['Time'].dt.hour*3600+dfCGM['Time'].dt.minute*60
    dfCGM['Abbot'].interpolate(method='linear',limit_direction='both',axis=0,inplace=True)
    dfCGM['Dexcom'].interpolate(method='linear',limit_direction='both',axis=0,inplace=True)
    dfCGM.to_csv(os.path.join(addDataPrefix,'All_cgm.csv'),index=False)
else:
    dfCGM=pd.read_csv(os.path.join(addDataPrefix,'All_cgm.csv'))
    dfCGM['Time']=dfCGM['Time'].astype('datetime64[ns]')

# mamad=dfCGM[dfCGM['Time']>dfCGM.iloc[5][1]]
# print(mamad)

In [14]:
if not os.path.exists(os.path.join(addDataPrefix,'All_cm.csv')):
    os.chdir(addHKCM)
    df=[]
    for root, dirs, files in os.walk(addHKCM):
        for file in files:
            if '.csv' in file.lower():
                if 'cm' in file.lower() and 'aaa' in file:
                    participantName=file[:file.find('_cm')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp = pd.read_csv(file)
                    print("File is read")
                    dfTemp.insert(0,'Participant',participantName)
                    print("column is added")
                    dfTemp['Date']=pd.to_datetime(dfTemp['Date'])
                    dfTemp['Date']=dfTemp['Date'].dt.tz_localize(None)
                    print("converted")
                    dfTemp.sort_values(['Date'],ascending = (True),inplace=True)
                    print("sorted")
                    dfTemp.rename(columns={'Date':'Time'}, inplace=True)
                    if len(dfTemp.columns)!=13:
                        print("MAYDAY. Error in reading csv")
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    print("reading is done")
    dfCM=df
    dfCM.drop("UID",axis=1,inplace=True)
    dfCM.drop("UnixTime",axis=1,inplace=True)

    # dfCM['Date']-=pd.DateOffset(hours=5)#fixing the time zone issue

    # dfCM.insert(2,'Time',dfCM['Date'])
    # dfCM['Date']=dfCM['Date'].dt.date
    # dfCM['Time']=dfCM['Time'].dt.hour*3600+dfCM['Time'].dt.minute*60+dfCM['Time'].dt.second+dfCM['Time'].dt.microsecond/1000/1000
    
    print("Processing is done")
    dfCM.to_csv(os.path.join(addDataPrefix,'All_cm.csv'),index=False)
else:
    dfCM=pd.read_csv(os.path.join(addDataPrefix,'All_cm.csv'))
    dfCM['Time']=dfCM['Time'].astype('datetime64[ns]')
    

Reading ... aaa_cm.csv
File is read
column is added
converted
sorted
reading is done
Processing is done


In [41]:
#Meal Format: Time, Carb, Fat, Protein to comply myfinesspal formating starting from Jan 31 til Feb 9
# breakFast=[[31,'07-52-34',170,12,15],
# [32,'07-09-13',170,12,60],
# [33,'07-19-09',170,48,15],
# [34,'06-47-38',170,12,15],
# [35,'07-27-52',170,48,60],
# [36,'07-18-46',42.5,12,15],
# [37,'07-18-46',170,12,15],
# [38,'07-44-07',170,12,60],
# [39,'07-23-33',170,48,15],
# [40,'06-50-00',170,12,15]]

# lunch=[[31,'12-20-32',170,12,15],
# [32,'12-19-09',170,12,60],
# [33,'12-17-27',42.5,12,60],
# [34,'12-11-35',170,12,60],
# [35,'11-57-05',42.5,12,60],
# [36,'12-35-21',170,48,60],
# [37,'12-08-40',42.5,12,60],
# [38,'11-22-23',170,12,15],
# [39,'12-16-25',42.5,12,60],
# [40,'12-12-34',170,48,15]]

# dinner=[[31,'17-55-02',84,50,27],
# [32,'17-21-14',49,28,41],
# [33,'17-50-56',21,28,53],
# [34,'17-45-17',70,35,39],
# [35,'17-18-12',110,28,46],
# [36,'17-41-39',53,51,67],
# [37,'16-44-56',68,39,25],
# [38,'15-58-07',71,52,82],
# [39,'16-59-39',66,43,28],
# [40,'17-44-56',122,47,51]]

# exercise=[[30,'10-10-00','11-02-00'],
#           [31,'16-07-00','17-10-00'],
#           [33,'15-10-00','16-03-00'],
#           [36,'15-30-00','16-23-00'],
#           [37,'09-49-00','11-06-00'],
#           [38,'09-34-00','10-34-00'],
#           [40,'14-50-00','15-47-00'],
#           [41,'16-12-00','17-07-00']]

# mealData=[]
# mealTemp=[]
# mealTemp.extend(breakFast)
# mealTemp.extend(lunch)
# # mealTemp.extend(dinner)


# for element in mealTemp:
#     timeTemp = datetime.strptime(element[1], '%H-%M-%S')
#     timeTemp=timeTemp.time()
#     timeTemp=timeTemp.hour*3600+timeTemp.minute*60+timeTemp.second
#     mealData.append([element[0],timeTemp,element[2],element[3],element[4]])

# dfMacro=pd.DataFrame(data=mealData,columns=['Date','Time','Carb','Fat','Protein'])

# activity=[]
# for element in exercise:
#     timeTempStart = datetime.strptime(element[1], '%H-%M-%S')
#     timeTempStart=timeTempStart.time()
#     timeTempStart=timeTempStart.hour*3600+timeTempStart.minute*60+timeTempStart.second

#     timeTempEnd = datetime.strptime(element[2], '%H-%M-%S')
#     timeTempEnd=timeTempEnd.time()
#     timeTempEnd=timeTempEnd.hour*3600+timeTempEnd.minute*60+timeTempEnd.second
#     activity.append([element[0],timeTempStart,timeTempEnd])
# dfActivity=pd.DataFrame(activity,columns=['Date','Start','End'])
# dfActivity.sort_values(by=['Date','Start'])

In [42]:
# def loadingDataReader(fileAddress,parameter):
#     try:
#         jsonFile = open(fileAddress, )
#         data = json.load(jsonFile)

#         dateList=[]
#         timeList=[]
#         valList=[]
#         for i in data:
#             for j in data[i]:
#                 timeTemp=j["time"]
#                 timeTemp=int(timeTemp[0:2])*3600+int(timeTemp[3:5])*60+int(timeTemp[6:8])
#                 timeList.append(timeTemp)

#                 dateTemp=i
#                 dateTemp = datetime.strptime(dateTemp, '%m-%d')+relativedelta(years=+121)
#                 dateTemp=dateTemp.timetuple().tm_yday
#                 dateList.append(dateTemp)
#                 valList.append(float(j["value"]))
#         jsonFile.close()
#     except:
#         print("An exception occurred")

#     parList = []
#     for i in range (0,len(dateList)):
#         parList.append(parameter)

#     return dateList,timeList,valList,parList

# def loadingDataAux(fileAddress,parameter,allDate,allTime,allVal,allPar):
#     fileAddress=os.path.normpath(fileAddress)
#     [tempDate,tempTime,tempVal,tempPar]=loadingDataReader(fileAddress,parameter)
#     allDate.extend(tempDate)
#     allTime.extend(tempTime)
#     allVal.extend(tempVal)
#     allPar.extend(tempPar)
#     print(parameter," is done")

#     return allDate,allTime,allVal,allPar

# def dataResEnhancer(df):
#     gapLength=1
#     df=df.sort_values(by=['Parameter', 'Date','Time'])
#     gapList=[]
#     highResSensor=['Acc','EDA','HR','Temp','CGM','Step','Cal']
#     for sensor in highResSensor:
#         newQuery=df[df['Parameter']==sensor]
#         for counter in range(0,len(newQuery)-1):
#             if newQuery.iloc[counter+1,0]==newQuery.iloc[counter+1,0]:
#                 if newQuery.iloc[counter+1,1]-newQuery.iloc[counter,1]>gapLength:
#                     gapList.append([newQuery.iloc[counter,0],newQuery.iloc[counter,1],newQuery.iloc[counter+1,1],newQuery.iloc[counter,3]])
#     appendedData=[]
#     for element in gapList:
#         duration= element[2]-element[1]
#         if duration%gapLength==0:
#             fillerNumber=int(duration/gapLength-1)
#         else:
#             fillerNumber=int(duration/gapLength)
#         for counter in range(0,fillerNumber):
#             appendedData.append([element[0],element[1]+(counter+1)*gapLength,float('nan'),element[3]])

#     df = df.append(pd.DataFrame(appendedData,columns=['Date','Time','Value','Parameter']),ignore_index = True)
#     df=df.sort_values(by=['Parameter','Date','Time'])

#     df=df.interpolate(method='linear')
#     return df

# def dataCleaner(df):
#     i = df[df.Date == 30].index #first data and noisy
#     df=df.drop(i)

#     i = df[df.Date == 40].index #Partial E4
#     df=df.drop(i)

#     i = df[df.Date == 41].index #No E4
#     df=df.drop(i)

#     i = df[df.Date >= 42].index #CGM becomes too noisy at the end of the study
#     df=df.drop(i)

#     return df

# def dataAmputator(df):
#     amputations=[[31,38290,39010],[31,61937,62638],[31,66226,71099],
#               [32,58667,59563],[32,63875,63902],
#               [33,25127,34731],[33,57996,58812],
#               [34,25556,30763],[34,58736,59877],[34,66069,66092],
#               [35,27945,31416],[35,57283,58147],
#               [36,29865,33926],[36,59231,60096],
#               [37,26911,32544],[37,40214,41380],
#               [38,26707,34560],[38,38172,39273],
#               [39,31053,35335],[39,57408,58655]]

#     for amputation in amputations:
#         i=df[(df['Date']==amputation[0]) & (df['Time']>=amputation[1]) & (df['Time']<=amputation[2])].index
#         df.loc[i,'Value']=float('nan')
#         # df=df.drop(i)
#     df=df.interpolate(method='linear')
#     return df

# def loadingData(addressPrefix,dfActivity):
#     if not os.path.exists(addressPrefix+'Result-interpolated.csv'):
#         allDate=[]
#         allTime=[]
#         allVal=[]
#         allPar=[]

#         sensorList=[['intraday-ACC_E4.json','Acc'],['intraday-calories.json','Cal'],
#                     ['intraday-EDA_E4.json','EDA'],['intraday-HR_E4.json','HR'],
#                     ['intraday-steps.json','Step'],['intraday-TEMP_E4.json','Temp'],
#                     ['intraday-glucose.json','CGM']]

#         for element in sensorList:
#             fileAddress=addressPrefix+element[0]
#             allDate,allTime,allVal,allPar=loadingDataAux(fileAddress,element[1],allDate,allTime,allVal,allPar)

#         dfOriginal = pd.DataFrame(list(zip(allDate,allTime,allVal,allPar)),
#                           columns =['Date','Time','Value','Parameter'])
#         dfOriginal=dfOriginal.sort_values(by=['Parameter','Date','Time'])
#         dfOriginal.to_csv(addressPrefix+'Result-original.csv', header=True,index=False)
#         print("Result-Original is saved")
#         dfInterp=dataCleaner(dfOriginal)
#         dfActivity=dataCleaner(dfActivity)
#         # dfInterp=dataResEnhancer(dfInterp)
#         dfInterp=dataAmputator(dfInterp)
#         dfInterp.to_csv(addressPrefix+'Result-interpolated.csv', header=True,index=False)
#         print("Result-Interpolated is saved")
#     else:
#         dfOriginal=pd.read_csv(addressPrefix+'Result-original.csv')
#         dfInterp=pd.read_csv(addressPrefix+'Result-interpolated.csv')
#     return dfOriginal,dfInterp,dfActivity

# dfOriginal,dfInterp,dfActivity=loadingData(addressPrefix,dfActivity)

In [43]:
# def activityExtractor(dfInterp):
#     data=[]
#     for i in range(len(dfActivity)):
#         currentDay=dfActivity.iloc[i,0]
#         dfTemp=dfInterp[dfInterp['Date']==currentDay]
#         dfTemp=dfTemp[dfTemp['Time']>=dfActivity.iloc[i,1]]
#         dfTemp=dfTemp[dfTemp['Time']<=dfActivity.iloc[i,1]+15*60]
#         for j in range(len(dfTemp)):
#             data.append([currentDay,dfTemp.iloc[j,1],dfTemp.iloc[j,2],dfTemp.iloc[j,3],1])
#     for i in range(len(dfActivity)):
#         currentDay=dfActivity.iloc[i,0]
#         dfTemp=dfInterp[dfInterp['Date']==currentDay]
#         dfTemp=dfTemp[dfTemp['Time']>=dfActivity.iloc[i,1]-15*60]
#         dfTemp=dfTemp[dfTemp['Time']<=dfActivity.iloc[i,1]]
#         for j in range(len(dfTemp)):
#             data.append([currentDay,dfTemp.iloc[j,1],dfTemp.iloc[j,2],dfTemp.iloc[j,3],0])
#     dfData=pd.DataFrame(data,columns=['Date','Time','Value','Parameter','ActivityFlag'])
#     return dfData

# dfInterp=dfInterp.loc[(dfInterp['Parameter']=='EDA') | (dfInterp['Parameter']=='HR') | (dfInterp['Parameter']=='Temp')]
# dfData=activityExtractor(dfInterp)
# dfData.to_csv(os.path.join(addressPrefix,'activity-tTest.csv'),index=False)


In [44]:
# def highResAccConvertor(data,startTime):
#     processedData=[]
#     for counter,row in enumerate(data):
#         if counter%2==0:
#             continue
#         tempTime=startTime+counter/32
#         tempTime=datetime.fromtimestamp(tempTime)
#         tempDay=tempTime.timetuple().tm_yday
#         tempDay=int(tempDay)
#         tempTime=tempTime.microsecond/1000000+tempTime.second+tempTime.minute*60+tempTime.hour*3600
#         axis=['AccX','AccY','AccZ']
#         for axe in range(0,3):
#             processedData.append([tempDay,tempTime,int(row[axe])/64,axis[axe]])
#     return processedData

# def highResAcc(addressPrefix):
#     addressPrefix=os.path.join(addressPrefix,'E4_Data/')
#     E4AccFiles=[]
#     data=[]
#     for root, dirs, files in os.walk(addressPrefix, topdown=False):
#        for name in files:
#            if 'ACC' in name:
#                E4AccFiles.append(os.path.join(root,name))
#     for element in E4AccFiles:
#         print(element)
#         rows = []
#         with open(element, 'r') as csvfile:
#             csvreader = csv.reader(csvfile)
#             fields = next(csvreader)
#             for row in csvreader:
#                 rows.append(row)
#         startTime=fields[0]
#         startTime=float(startTime)
#         startTime=int(startTime)
#         data.extend(highResAccConvertor(rows,startTime))
#     df = pd.DataFrame(data,columns=['Date','Time','Value','Parameter'])
#     df=df.sort_values(by=['Date','Time','Parameter'])
#     df.to_csv(os.path.join(addressPrefix,'AccHighRes'))
#     return df

# if os.path.exists(os.path.join(addressPrefix,'AccHighRes.csv')):
#     dfAccHighRes=pd.read_csv(os.path.join(addressPrefix,'AccHighRes.csv'))
# else:
#     dfAccHighRes=highResAcc(addressPrefix)
# print(dfAccHighRes)

In [45]:
# def XGClassifier(dataList, labelList,randomSeed):
#     trainData, testData, trainLabels, testLabels = train_test_split(dataList, labelList, test_size=0.33,random_state=randomSeed)
#     accuracyBest=0
#     for maxDepth in np.arange(3,30):
#         for estimator in np.arange(5,50,2):
#             clf = xgb.XGBClassifier(n_estimators=estimator,max_depth=maxDepth,objective = "binary:logistic",eval_metric = "logloss",use_label_encoder =False)
#             clf.fit(trainData, trainLabels)
#             slidingWindowPrediction = clf.predict_proba(testData)
#             slidingWindowPrediction=slidingWindowPrediction[:,1]
#             slidingWindowPrediction[slidingWindowPrediction>=0.5]=1
#             slidingWindowPrediction[slidingWindowPrediction<0.5]=0

#             confMatrix=sklearn.metrics.confusion_matrix(testLabels,slidingWindowPrediction)
#             accuracy=sklearn.metrics.accuracy_score(testLabels,slidingWindowPrediction)
#             recall=sklearn.metrics.recall_score(testLabels,slidingWindowPrediction)
#             precision=sklearn.metrics.precision_score(testLabels,slidingWindowPrediction)

#             if accuracy>accuracyBest:
#                 confMatrixBest=confMatrix
#                 accuracyBest=accuracy
#                 modelBest=clf
#                 recallBest=recall
#                 precisionBest=precision
#     # print(confMatrixBest)
#     print(np.round(100*accuracyBest,0),np.round(100*recallBest,0),np.round(100*precisionBest,0))

# def statAnalyzer(dfInterp,sensors):
#     print(sensors)
#     dfMeal=pd.DataFrame([],columns=['Date','Time','Value','Parameter'])
#     dfBeforeMeal=pd.DataFrame([],columns=['Date','Time','Value','Parameter'])
#     days=dfInterp['Date'].to_list()

#     days=list(set(days))
#     mealList=[]
#     mealList.extend(breakFast)
#     mealList.extend(lunch)
#     mealList.extend(dinner)

#     dataList=[]
#     labelList=[]
#     for mealCat in range(0,2):
#         for element in mealList:
#             sensorData=[]
#             for sensor in sensors:
#                 mealTime=datetime.strptime(element[1], '%H-%M-%S')
#                 mealTime=mealTime.time().second+mealTime.time().minute*60+mealTime.time().hour*3600

#                 dfTemp=dfInterp[dfInterp['Date']==element[0]]
#                 dfTemp=dfTemp[dfTemp['Parameter']==sensor]
#                 dfTemp=dfTemp[dfTemp['Time']>mealTime-mealCat*20*60]
#                 dfTemp=dfTemp[dfTemp['Time']<=mealTime+(1-mealCat)*20*60]
#                 dfTemp['Value']=dfTemp['Value']
#                 dfTemp.insert(4,'mealMoment',1)
#                 dfMeal = dfMeal.append(dfTemp, ignore_index=True)

#                 tempData=dfTemp['Value'].to_list()
#                 sensorData.extend(tempData)
#             labelList.append(1-mealCat)
#             dataList.append(sensorData)

#     dfMeal=dfMeal.append(dfBeforeMeal,ignore_index=True)
#     dfMeal=dfMeal.sort_values(by=['Parameter', 'Date','Time'])

#     dataListNew=[]
#     labelListNew=[]

#     for counter in range(0,len(dataList)):
#         if len(dataList[counter])==len(sensors)*1200:
#             dataListNew.append(dataList[counter])
#             labelListNew.append(labelList[counter])

#     dataList=np.asarray(dataListNew)
#     labelList=np.asarray(labelListNew)

#     return dfMeal,dataList,labelList

# sensorList=[['EDA','Temp'],['EDA'],['Temp']]
# for randomSeed in np.arange(10,100,10):
#     for sensors in sensorList:
#         dfMeal,dataList,labelList=statAnalyzer(dfInterp,sensors)
#         XGClassifier(dataList, labelList,randomSeed)
#     print('------------------')
# dfMeal.to_csv('lunch.csv',index=False)


In [39]:
def featureExtractorMotion(df):
    f1=0
    f2=0
    f3=0
    for counter in range(len(df)):
        temp1=abs(df['Yaw'][counter])+abs(df['Pitch'][counter])+abs(df['Roll'][counter])
        temp2=abs(df['Ax'][counter])+abs(df['Ay'][counter])+abs(df['Az'][counter])
        f2+=temp2
        f3+=temp1
        if(temp2==0):
            temp2+=0.001#We had to truncate the sensor data in the transmition process. We are adding epsilon to fix it
        f1+=temp1/temp2
    f1/=len(df)
    f2/=len(df)
    return [f1,f2]

def featureExtractorCGM(df):
    mean=df['Abbot'].mean()
    std=df['Abbot'].std()
    max=df['Abbot'].max()
    min=df['Abbot'].min()
    return mean, std, min,max

MINIMUM_POINT_CM=10
WINDOW_LENGTH=timedelta(minutes=1)
WINDOW_STEP=timedelta(minutes=1)
MEAL_PORTION_RELAXATION=timedelta(seconds=10)

participants=dfMeal['Participant'].to_list()
participants=list(set(participants))
startofTrial = datetime.strptime('11 06 2021-13:58:00', '%m %d %Y-%H:%M:%S')
endofTrial = datetime.strptime('11 15 2021-00:00:00', '%m %d %Y-%H:%M:%S')

windowStart=startofTrial
windowEnd=windowStart+WINDOW_LENGTH
for participant in participants:
    participantList=[]
    while(windowStart<endofTrial):
        dfTempCM=dfCM.loc[(dfCM['Time']>=windowStart) & (dfCM['Time']<=windowEnd)& dfCM['Participant']==participant]
        dfTempMeal=dfMeal.loc[(dfMeal['StartTime']-MEAL_PORTION_RELAXATION>=windowStart) & (dfTempMeal['FinishTime']+MEAL_PORTION_RELAXATION<=windowEnd)& (dfCM['Participant']==participant)]
        if(len(dfTempCM)<MINIMUM_POINT_CM):
            windowStart+=WINDOW_STEP
            windowEnd+=WINDOW_STEP
            continue
        tempList=featureExtractorMotion(dfTempCM)
        if(len(dfTempMeal)!=0):
            tempList.append(1)
        else:
            tempList.append(0)
        participantList.append(tempList)
        windowStart+=WINDOW_STEP
        windowEnd+=WINDOW_STEP  
    break
print(participantList)


[]


In [46]:
def STMI_XGBoost(trainData,testData,trainLabels,testLabels):
    mseBest=math.inf
    depthBest=float('nan')
    estimatorBest=float('nan')
    modelBest=''

    for numEstimator in np.arange(10,120,10):
        for maxDepth in np.arange(3,10):
            clf = xgb.XGBRegressor(n_estimators =numEstimator,max_depth=maxDepth,objective ='reg:squarederror',eval_metric = "logloss")
            clf.fit(trainData, trainLabels)
            testPrediction = clf.predict(testData)
            mre=[]
            for counter in range(0,len(testPrediction)):
                mre.append(math.fabs(testPrediction[counter]-testLabels[counter])/testLabels[counter])
            mre=np.asarray(mre)
            mre=np.mean(mre)
            if mre<mseBest:
                mreBest=mre
                depthBest=maxDepth
                estimatorBest=numEstimator
                modelBest=clf
                predictionBest=testPrediction
    print(clf.feature_importances_)
    return modelBest,mreBest,predictionBest,depthBest,estimatorBest

def aucCalculator(timeTemp,dataTemp,kernelNumber):
    kernelMeans=np.linspace(0,24,kernelNumber+2)
    kernelMeans=np.delete(kernelMeans,0)
    kernelMeans=np.delete(kernelMeans,-1)
    kernelMeans=kernelMeans*3600

    kernelSTD=2*3600
    aucData=[]
    for kernelMean in kernelMeans:
        kernelTemp=np.exp(-((timeTemp-kernelMean)**2)/(2*kernelSTD**2))
        tempAuc=kernelTemp*dataTemp
        tempAuc=np.trapz(tempAuc,x=timeTemp)
        aucData.append(tempAuc/3600)

    return aucData

def statisticalFeature(timeTemp,dataTemp):
    featureData=[]
    featureList=[]

    featureData.append(np.mean(dataTemp))
    featureData.append(np.max(dataTemp)-np.min(dataTemp))
    featureData.append(np.var(dataTemp))

    featureData.append(np.mean(np.diff(dataTemp,n=1)))
    # featureData.append(np.median(np.diff(dataTemp,n=1)))
    featureData.append(np.max(np.diff(dataTemp,n=1)))
    featureData.append(np.min(np.diff(dataTemp,n=1)))
    # featureData.append(np.var(np.diff(dataTemp,n=1)))

    featureData.append(np.mean(np.diff(dataTemp,n=2)))
    # featureData.append(np.median(np.diff(dataTemp,n=2)))
    # featureData.append(np.max(np.diff(dataTemp,n=2)))
    # featureData.append(np.min(np.diff(dataTemp,n=2)))
    # featureData.append(np.var(np.diff(dataTemp,n=2)))

    featureData.append(np.trapz(dataTemp)/3600)
    return featureData

def featureExtractor(df,contextFlag):
    featureList=[]
    sensors=['CGM','EDA', 'Acc','Temp', 'HR','Cal']


    #-----------------I removed the STEP as it was almost always zero. We should look into it later------------
    featureLabels=[]
    for sensor in sensors:

        varData=df[df['Parameter']==sensor]
        varTime=varData['Time'].tolist()
        varData=varData['Value'].tolist()

        varTime=np.asarray(varTime)
        varData=np.asarray(varData)
        if contextFlag:
            featureList.extend(statisticalFeature(varTime,varData))
            featureLabels.append([sensor,'mean'])
            featureLabels.append([sensor,'max'])
            featureLabels.append([sensor,'var'])
            featureLabels.append([sensor,'S-mean'])
            featureLabels.append([sensor,'S-max'])
            featureLabels.append([sensor,'S-min'])
            featureLabels.append([sensor,'SS-mean'])
            featureLabels.append([sensor,'trapz'])
        if sensor=='CGM':
            featureList.extend(aucCalculator(varTime,varData,9))
            for i in range(0,9):
                featureLabels.append([sensor,'auc'+str(i)])
            featureList.extend(aucCalculator(varTime,varData,6))
            for i in range(0,6):
                featureLabels.append([sensor,'auc'+str(i)])
    print("Used features",featureLabels)
    #REDO IT WITH FITBIT AND E4
    #FOR MEAL TO MEAL PERSPECTIVE, MAKE THE WHOLE DAY INTO 3 CHUNCKS (MEAL TO MEAL)
    #PLOT DAY-DAY VS MEAL TO MEAL AND SHOW THEM HOW WE ARE SEAPRATING THINGS (CGM,EDA,...)

    #https://github.com/fraunhoferportugal/tsfel/tree/9319db4368303cf10adb3aeb72cd4235a8085307
    return featureList

def testTrainSplitFuncAux(dfSensorTemp,dfMacroTemp,contextFlag):
    sensorDataList=[]
    macroDataList=[]
    days=list(set(dfSensorTemp['Date']))

    for day in days:
        dfSensorTempDay=dfSensorTemp[dfSensorTemp['Date']==day]
        sensorDataList.append(featureExtractor(dfSensorTempDay,contextFlag))
        dfMacroTempDay=dfMacroTemp[dfMacroTemp['Date']==day]
        totalCarb=dfMacroTempDay['Carb'].sum()
        totalFat=dfMacroTempDay['Fat'].sum()
        totalProtein=dfMacroTempDay['Protein'].sum()
        macroDataList.append([totalCarb,totalFat,totalProtein])

    return sensorDataList,macroDataList

def featureNormalizer(trainData,testData):
    # repeatingNum=trainData.shape
    # repeatingNum=repeatingNum[0]
    #
    # columnMeans=np.mean(trainData,axis=0)
    # columnMeans = np.repeat(columnMeans[:, np.newaxis], repeatingNum, axis=1)
    # columnMeans=np.transpose(columnMeans)
    # trainData-=columnMeans
    #
    # columnSTD=np.std(trainData,axis=0)
    # columnSTD = np.repeat(columnSTD[:, np.newaxis], repeatingNum, axis=1)
    # columnSTD=np.transpose(columnSTD)
    # trainData/=columnSTD
    #
    # repeatingNum=testData.shape
    # repeatingNum=repeatingNum[0]
    #
    # columnMeans=np.mean(trainData,axis=0)
    # columnMeans = np.repeat(columnMeans[:, np.newaxis], repeatingNum, axis=1)
    # columnMeans=np.transpose(columnMeans)
    # testData-=columnMeans
    #
    # columnSTD=np.std(trainData,axis=0)
    # columnSTD = np.repeat(columnSTD[:, np.newaxis], repeatingNum, axis=1)
    # columnSTD=np.transpose(columnSTD)
    # testData/=columnSTD
    scaler = StandardScaler()
    trainData = scaler.fit_transform(trainData)
    testData = scaler.fit_transform(testData)

    return trainData,testData

def testTrainSplitFunc(dfSensor, dfMacro,randomSeed,contextFlag):
    days=list(set(dfSensor['Date']))
    random.seed(randomSeed)
    random.shuffle(days)
    daysTrain=days[0:6]
    daysTest=days[6:9]

    dfSensorTrain = dfSensor[dfSensor['Date'].isin(daysTrain)]
    dfSensorTest = dfSensor[dfSensor['Date'].isin(daysTest)]

    dfMacroTrain= dfMacro[dfMacro['Date'].isin(daysTrain)]
    dfMacroTest= dfMacro[dfMacro['Date'].isin(daysTest)]

    trainData,trainLabel=testTrainSplitFuncAux(dfSensorTrain,dfMacroTrain,contextFlag)
    testData,testLabel=testTrainSplitFuncAux(dfSensorTest,dfMacroTest,contextFlag)

    trainData=np.asarray(trainData)
    testData=np.asarray(testData)

    trainLabel=np.asarray(trainLabel)
    testLabel=np.asarray(testLabel)

    trainLabel=trainLabel[:,0]
    testLabel=testLabel[:,0]
    trainData,testData=featureNormalizer(trainData,testData)
    # for i in range(0,testData.shape[1]):
    #     plt.figure()
    #     plt.plot(testData[:,i])
    #     plt.title(str(i))
    return trainData,testData,trainLabel,testLabel

randomSeed=random.randrange(100)
trainData,testData,trainLabel,testLabel=testTrainSplitFunc(dfInterp,dfMacro,randomSeed=randomSeed,contextFlag=True)
_,mreCGMCon,predictionCGMCon,_,_=STMI_XGBoost(trainData,testData,trainLabel,testLabel)
pearsonCGMCon=pearsonr(predictionCGMCon,testLabel)
pearsonCGMCon=pearsonCGMCon[0]

trainData,testData,trainLabel,testLabel=testTrainSplitFunc(dfInterp,dfMacro,randomSeed=randomSeed,contextFlag=False)
_,mreCGM,predictionCGM,_,_=STMI_XGBoost(trainData,testData,trainLabel,testLabel)
pearsonCGM=pearsonr(predictionCGM,testLabel)
pearsonCGM=pearsonCGM[0]

plt.scatter(x=testLabel,y=predictionCGM,color='red',label='CGM')
plt.scatter(x=testLabel,y=predictionCGMCon,color='blue',label='CGM+Context')
plt.plot([200,600],[200,600],':k')
# plt.scatter(x=daysTest,y=predictionBestCGMContext,label='Prediction CGM+Con')
# plt.scatter(x=daysTest,y=predictionBestCGM,label='Prediction CGM')
plt.annotate('CGM Error'+str(round(100*mreCGM,2))+'%', xy=(1.05, 0.95), xycoords='axes fraction')
plt.annotate('CGMCon Error'+str(round(100*mreCGMCon,2))+'%', xy=(1.05, 0.85), xycoords='axes fraction')

plt.annotate('CGM Pearson'+str(round(100*pearsonCGM,2))+'%', xy=(1.05, 0.75), xycoords='axes fraction')
plt.annotate('CGMCon Pearson'+str(round(100*pearsonCGMCon,2))+'%', xy=(1.05, 0.65), xycoords='axes fraction')

plt.xlabel('Actual Carb [gr]')
plt.ylabel('Predicted Carb [gr]')
plt.legend(loc='upper left')
plt.savefig('C:\GitHub\Carb.png')

NameError: name 'dfInterp' is not defined